# <font color=4CBB17>**PROYECTO KAGGLE**</font>

<font color=FF7F50><h3><b>Estudiante:</b></font> Mateo Toro López</h3>
<font color=FF7F50><h3><b>Programa:</b></font> Ingeniería Industrial</h3>


### <font color=1E90FF>**Descripción**</font>

Las **Pruebas Saber Pro** son exámenes estandarizados que se administran en Colombia para evaluar la calidad y el nivel de conocimiento y competencias de los estudiantes de educación superior, es decir, de instituciones de educación superior como universidades y tecnológicos. Estas pruebas son parte de los esfuerzos del **Gobierno de Colombia** para monitorear y mejorar la calidad de la educación superior en el país.

Estas Pruebas constan de cinco componentes genéricos: **Inglés, Lectura Crítica, Competencias Ciudadanas, Razonamiento Cuantitativo y Comunicación Escrita**.

El trabajo será crear un modelo de clasificación que, para cada estudiante, prediga qué desempeño va a tener: **bajo, medio-bajo, medio-alto o alto**.


## <font color=4CBB17>**Librerias**</font>


In [1]:
# Datos
import pandas as pd
import numpy as np
from itertools import product

## Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

## Modelado

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import statsmodels.api as sm
from sklearn.metrics import classification_report
import math


from collections import Counter
import time

# Para modelo Random Forest
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score



# <font color=4CBB17>**Descargar la base de datos desde Kaggle**</font>


### <font color=1E90FF>**Instrucciones**</font>

**1.** Crea un archivo kaggle.json con tu token de autenticación (en kaggle → haz clic en el ícono de usuario en la esquina superior derecha → configuración → API crear nuevo token)

**2.** Subir el archivo kaggle.json a este espacio de trabajo de Colab ⬇️

In [2]:
#Subir el archivo kaggle.json a este espacio de trabajo de Colab
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


**3.** Correr la sigiente celda ⬇️

In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 84% 17.0M/20.1M [00:01<00:00, 27.3MB/s]
100% 20.1M/20.1M [00:01<00:00, 19.2MB/s]


### <font color=1E90FF>**Descomprimir e inspeccionar datos**</font>


In [4]:
!unzip udea*.zip > /dev/null

In [5]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


### <font color=1E90FF>**cargar `train.csv` con pandas**</font>


In [6]:
# Librerias para procesamiento
import pandas as pd
import numpy as np

df = pd.read_csv("train.csv")

In [7]:
# Cuantas filas y columnas tiene el dataframe
rows, columns = df.shape
print(f"Número de filas: {rows}")
print(f"Número de columnas: {columns}")

Número de filas: 692500
Número de columnas: 12


In [8]:
df.set_index('ID', inplace=True)

In [9]:
df1 = df.copy() # Se crea una copia del dataframe original


# <font color=4CBB17>**Preprocesado**</font>


In [10]:
df1.PERIODO.value_counts()

,count
PERIODO,
20195,180873
20203,171838
20212,171412
20183,164818
20194,1472
20213,1178
20202,490
20184,254
20196,165


In [11]:
# Se crea una nueva columna llamada AÑO ya que hay unas categorias muy grandes y las otras demasiado pequeñas
df1['PERIODO'] = df1['PERIODO'].astype(str)
df1['AÑO'] = ['2018' if i in ['20183', '20184']
              else '2019' if i in ['20195', '20194', '20196']
              else '2020' if i in ['20203', '20202']
              else '2021' if i in ['20212', '20213']
              else i for i in df1['PERIODO']]

In [12]:
df1.AÑO.value_counts()

,count
AÑO,
2019,182510
2021,172590
2020,172328
2018,165072


>La variable PERIODO no tenía datos nulos, por lo tanto no fue necesario reemplazarlos, se creo la nueva variable AÑO ya que los periodos no estaban completos

In [13]:
# Asignar a los valores nulos "No info" para variables categoricas educación de la madre y padre

df1.FAMI_EDUCACIONMADRE.values[df1.FAMI_EDUCACIONMADRE.isna()] = 'No info'

df1.FAMI_EDUCACIONPADRE.values[df1.FAMI_EDUCACIONPADRE.isna()] = 'No info'


sum(df1.FAMI_EDUCACIONMADRE.isna()), sum(df1.FAMI_EDUCACIONPADRE.isna())

(0, 0)

In [14]:
# Se reeemplazan las categorias 'No sabe', 'No Aplica', 'No info' en una nueva llamada indeterminado
df1['FAMI_EDUCACIONMADRE'] = ['Indeterminado' if i in ['No sabe', 'No Aplica', 'No info'] else i for i in df1['FAMI_EDUCACIONMADRE'].values]
df1['FAMI_EDUCACIONPADRE'] = ['Indeterminado' if i in ['No sabe', 'No Aplica', 'No info'] else i for i in df1['FAMI_EDUCACIONPADRE'].values]

In [15]:
df1.FAMI_EDUCACIONPADRE.value_counts()

,count
FAMI_EDUCACIONPADRE,
Secundaria (Bachillerato) completa,128289
Primaria incompleta,125675
Educación profesional completa,83117
Secundaria (Bachillerato) incompleta,71654
Técnica o tecnológica completa,62995
Primaria completa,55958
Indeterminado,48999
Postgrado,44169
Educación profesional incompleta,27084


In [16]:
df1.FAMI_EDUCACIONMADRE.value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,141744
Primaria incompleta,99420
Técnica o tecnológica completa,89542
Educación profesional completa,85326
Secundaria (Bachillerato) incompleta,81012
Primaria completa,56125
Postgrado,46246
Indeterminado,28599
Técnica o tecnológica incompleta,27533


In [17]:
# Asignar a los valores nulos "No info" en la variable del Valor matricula Universidad
df1.ESTU_VALORMATRICULAUNIVERSIDAD.values[df1.ESTU_VALORMATRICULAUNIVERSIDAD.isna()] = 'No info'


In [18]:
# Se asigna el promedio de pago de matricula para cada categoria dentro de la variable

cmap = {'Entre 1 millón y menos de 2.5 millones': 1.75,
    'Entre 2.5 millones y menos de 4 millones': 3.25,
    'Menos de 500 mil': .250,
    'Entre 500 mil y menos de 1 millón': .75,
    'Entre 4 millones y menos de 5.5 millones': 4.75,
    'Más de 7 millones': 7.75,
    'Entre 5.5 millones y menos de 7 millones': 6.25,
    'No pagó matrícula': 0,
    'No info': -1}

# Usar map para transformar los valores
df1['ESTU_VALORMATRICULAUNIVERSIDAD'] = df1['ESTU_VALORMATRICULAUNIVERSIDAD'].map(cmap)

# Contar los valores únicos
df1['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
1.75,204048
3.25,127430
0.25,80263
0.75,78704
4.75,69736
7.75,68014
6.25,38490
0.00,19528
-1.00,6287


In [19]:
counts = df1['ESTU_PRGM_DEPARTAMENTO'].value_counts()

umbral = 2000
categorias_a_mantener = counts[counts > umbral].index

# Agrupar las categorías menos frecuentes como "Otros"
df1['ESTU_PRGM_DEPARTAMENTO'] = df1['ESTU_PRGM_DEPARTAMENTO'].apply(lambda x: x if x in categorias_a_mantener else 'Zonas dificil acceso')


In [20]:
df1['ESTU_PRGM_DEPARTAMENTO'].value_counts()

,count
ESTU_PRGM_DEPARTAMENTO,
BOGOTÁ,282159
ANTIOQUIA,83607
VALLE,44588
ATLANTICO,41020
SANTANDER,28828
NORTE SANTANDER,22588
BOLIVAR,20629
BOYACA,14048
CUNDINAMARCA,14018


In [21]:
# Se verifica nuevamente nulos en la variable ESTU_HORASSEMANATRABAJA
df1.ESTU_HORASSEMANATRABAJA.isna().sum()

30857

In [22]:
## Asignar a los valores nulos "No info" para variable de horas que trabaja el estudiante a la semana
df1.ESTU_HORASSEMANATRABAJA.values[df1.ESTU_HORASSEMANATRABAJA.isna()] = 'No info'
sum(df1.ESTU_HORASSEMANATRABAJA.isna())

0

In [23]:
# Se asigna el promedio de horas para cada categoria dentro de la variable

emap = {'0': 0,
    'Menos de 10 horas': 5,
    'Entre 11 y 20 horas': 15.5,
    'Entre 21 y 30 horas': 25.5,
    'Más de 30 horas': 35.5,
    'No info': -1}

# Usar map para transformar los valores
df1['ESTU_HORASSEMANATRABAJA'] = df1['ESTU_HORASSEMANATRABAJA'].map(emap)

# Contar los valores únicos
df1['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
35.5,249352
0.0,116550
15.5,115857
25.5,92693
5.0,87191
-1.0,30857


In [24]:
df1['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
35.5,249352
0.0,116550
15.5,115857
25.5,92693
5.0,87191
-1.0,30857


In [25]:
df1.FAMI_ESTRATOVIVIENDA.values[df.FAMI_ESTRATOVIVIENDA.isna()] = 'No info'
sum(df1.FAMI_ESTRATOVIVIENDA.isna())

0

In [26]:
df1['FAMI_ESTRATOVIVIENDA'].value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
No info,32137
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [27]:
# Mapeo de carreras
categoria_map = {
    'Ingeniería Industrial': ['INGENIERIA INDUSTRIAL', 'INGENIERÍA INDUSTRIAL'],
    'Otras Ingenierías': ['INGENIERIA', 'INGENIERÍA', 'INGE'],
    'Licenciaturas': ['LICENCIATURA',],
    'Exactas': ['QUIMICA', 'ESTAD', 'BIOLOG', 'ANTRO', 'MATEMATICAS', 'FISICA', 'GEOLO'],
    'Lenguas':['INGLES', 'LITER', 'LENGUA', 'IDIOMA'],
    'Ciencias Economicas':['CONTADURÍA', 'CONTADURIA', 'ECONOM','FINAN'],
    'Administracion de empresas': ['ADMINISTRACION DE EMPRESAS', 'ADMINISTRACIÓN DE EMPRESAS', 'ADMINISTRACION DE NEGOCIOS INTERNACIONALES'],
    'Otras Admin':  ['ADMINISTRACION', 'ADMINISTRACIÓN'],
    'Animales': ['ZOOTECNIA', 'VETERINARIA'],
    'Salud': ['ENFERM', 'ODONTO','SALUD', 'QUIR', 'FISIO', 'NUTRI', 'FONOAUDIO' ],
    'Ciencias Sociales': ['PSICO', 'COMUNICA','SOCIAL','PERIO','HIST', 'SOCIO', 'FILOSO', 'TEOLO'],
    'Negocios': ['NEGOCIOS', 'MERCADEO', 'MARKETING', 'PUBLICIDAD', 'INTERNAC'],
    'Derecho': ['DERECHO'],
    'Ciencias Politicas':['CIENCIA POLITICA', 'POLITICA','POLÍTICA', 'JURISPRUDENCIA'],
    'Medicina':['MEDICINA'],
    'Arquitectura y Diseño':['ARQUITECTURA', 'DISEÑO'],
    'Entretenimiento':['ARTE', 'ARTES', 'MUSICA','MÚSICA' 'CINE', 'TV','TEATRO']
}

# Clasificación de las carreras usando apply y búsqueda de key words
df1['PRGM_AREA'] = df1['ESTU_PRGM_ACADEMICO'].apply(lambda carrera:
    next((categoria for categoria, keywords in categoria_map.items()
          if any(keyword in carrera.upper() for keyword in keywords)),
         'Otras'))

In [28]:
df1['PRGM_AREA'].value_counts()

,count
PRGM_AREA,
Otras Ingenierías,113640
Ciencias Sociales,85293
Administracion de empresas,84790
Ciencias Economicas,72707
Licenciaturas,66280
Derecho,53545
Otras Admin,35016
Ingeniería Industrial,34794
Salud,34621


In [29]:
## Asignar a los valores nulos "No info" para variable de estrato
df1.FAMI_TIENEINTERNET.values[df.FAMI_TIENEINTERNET.isna()] = 'No info'
sum(df1.FAMI_TIENEINTERNET.isna())
df1['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
Si,592514
No,73357
No info,26629


In [30]:
# Mapeo de valores
map = {
    'Si': 1,
    'No': 0,
    'No info': -1
}

# Aplicar el mapeo a la columna
df1['FAMI_TIENEINTERNET'] = df1['FAMI_TIENEINTERNET'].map(map)

In [31]:
## Asignar a los valores nulos "No info" para variable de estrato
df1.ESTU_PAGOMATRICULAPROPIO.values[df.ESTU_PAGOMATRICULAPROPIO.isna()] = 'No info'
sum(df1.ESTU_PAGOMATRICULAPROPIO.isna())


0

In [32]:
# Mapeo de valores
mapping = {
    'Si': 1,
    'No': 0,
    'No info': -1
}

# Aplicar el mapeo a la columna
df1['ESTU_PAGOMATRICULAPROPIO'] = df1['ESTU_PAGOMATRICULAPROPIO'].map(mapping)


In [33]:
df1['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
0,382201
1,303801
-1,6498


In [34]:
# Se elimina variable respuesta y dos variables categoricas que no se van a usar en el modelo
X_features = df1.drop(['RENDIMIENTO_GLOBAL','PERIODO', 'ESTU_PRGM_ACADEMICO'], axis=1, inplace=False)

In [35]:
X_features.dtypes

,0
ESTU_PRGM_DEPARTAMENTO,object
ESTU_VALORMATRICULAUNIVERSIDAD,float64
ESTU_HORASSEMANATRABAJA,float64
FAMI_ESTRATOVIVIENDA,object
FAMI_TIENEINTERNET,int64
FAMI_EDUCACIONPADRE,object
FAMI_EDUCACIONMADRE,object
ESTU_PAGOMATRICULAPROPIO,int64
AÑO,object
PRGM_AREA,object


In [72]:
# Estrato es categorica ordinal (ya que se puede establecer un orden), por lo tanto se le asigna dicho orden
dic = {'No info':-1, 'Sin Estrato':0, 'Estrato 1':1, 'Estrato 2':2,  'Estrato 3':3,  'Estrato 4':4, 'Estrato 5':5, 'Estrato 6':6}
X_features['FAMI_ESTRATOVIVIENDA'] = X_features['FAMI_ESTRATOVIVIENDA'].replace(dic).infer_objects()


In [37]:
X_features['FAMI_ESTRATOVIVIENDA'] = X_features['FAMI_ESTRATOVIVIENDA'].astype('int')

In [38]:

columnas_a_transformar = ['FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE', 'AÑO','PRGM_AREA', 'ESTU_PRGM_DEPARTAMENTO']
columnas_a_escalar = ['FAMI_TIENEINTERNET','ESTU_PAGOMATRICULAPROPIO','ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA']

In [39]:
# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [40]:
# Dumización de variables independientes
columnas_a_transformar=pd.get_dummies(X_features[columnas_a_transformar])

In [41]:
from sklearn.preprocessing import StandardScaler

# Escalado de las variables categoricas a one hot
X_features[columnas_a_escalar] = StandardScaler().fit_transform(X_features[columnas_a_escalar])

In [42]:
# Seleccionar las columnas no transformadas
escaladas = X_features.drop(columns= ['FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE','AÑO','PRGM_AREA', 'ESTU_PRGM_DEPARTAMENTO'])

# Concatenar las columnas transformadas y las no transformadas
X_features_final = pd.concat([escaladas, columnas_a_transformar], axis=1)

X_features_final.head(3)

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONPADRE_Educación profesional completa,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Indeterminado,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Indeterminado,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,AÑO_2018,AÑO_2019,AÑO_2020,AÑO_2021,PRGM_AREA_Administracion de empresas,PRGM_AREA_Animales,PRGM_AREA_Arquitectura y Diseño,PRGM_AREA_Ciencias Economicas,PRGM_AREA_Ciencias Politicas,PRGM_AREA_Ciencias Sociales,PRGM_AREA_Derecho,PRGM_AREA_Entretenimiento,PRGM_AREA_Exactas,PRGM_AREA_Ingeniería Industrial,PRGM_AREA_Lenguas,PRGM_AREA_Licenciaturas,PRGM_AREA_Medicina,PRGM_AREA_Negocios,PRGM_AREA_Otras,PRGM_AREA_Otras Admin,PRGM_AREA_Otras Ingenierías,PRGM_AREA_Salud,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_Zonas dificil acceso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
904256,1.482661,-0.994623,0.485119,0.384334,-0.835923,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
645256,0.191253,-1.340601,0.485119,-1.717718,-0.835923,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
308367,0.191253,1.115843,0.485119,0.384334,-0.835923,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [43]:
# Separar la variable respuesta o dependiente para luego convertir la columna de target (predicción) en valores discretos
y_target = 'RENDIMIENTO_GLOBAL'

rmap = {'alto': 3, 'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2}

# Usar map para transformar los valores
df1[y_target] = df1[y_target].map(rmap)


In [44]:
# Grafico variable respuesta

counts = df1[y_target].value_counts().reset_index()
counts.columns = [y_target, 'Count']  # Renombrar las columnas

counts['Percentage'] = (counts['Count'] / counts['Count'].sum()) * 100
counts['Label'] = counts['Count'].astype(str) + ' (' + counts['Percentage'].round(1).astype(str) + '%)'
print(f'Gráfica')

fig = px.bar(counts, x=y_target, y='Count', color=y_target, title=f'Distribución de {y_target}',
             labels={y_target:'Rendimiento global', 'Count': 'Frecuencia'}, text='Label', width=1000, height=600)

fig.show()

Gráfica


In [45]:
counts

,RENDIMIENTO_GLOBAL,Count,Percentage,Label
0,3,175619,25.360144,175619 (25.4%)
1,0,172987,24.980072,172987 (25.0%)
2,1,172275,24.877256,172275 (24.9%)
3,2,171619,24.782527,171619 (24.8%)


In [46]:
# Variable respuesta
df1[y_target]

,RENDIMIENTO_GLOBAL
ID,
904256,2
645256,0
308367,0
470353,3
989032,1
...,...
25096,2
754213,0
504185,1


In [47]:
# Variable respuesta
y_target

'RENDIMIENTO_GLOBAL'

In [48]:
# Variables regresoras o independientes
X_features_final.head(3)

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONPADRE_Educación profesional completa,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Indeterminado,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Indeterminado,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta,AÑO_2018,AÑO_2019,AÑO_2020,AÑO_2021,PRGM_AREA_Administracion de empresas,PRGM_AREA_Animales,PRGM_AREA_Arquitectura y Diseño,PRGM_AREA_Ciencias Economicas,PRGM_AREA_Ciencias Politicas,PRGM_AREA_Ciencias Sociales,PRGM_AREA_Derecho,PRGM_AREA_Entretenimiento,PRGM_AREA_Exactas,PRGM_AREA_Ingeniería Industrial,PRGM_AREA_Lenguas,PRGM_AREA_Licenciaturas,PRGM_AREA_Medicina,PRGM_AREA_Negocios,PRGM_AREA_Otras,PRGM_AREA_Otras Admin,PRGM_AREA_Otras Ingenierías,PRGM_AREA_Salud,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_Zonas dificil acceso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
904256,1.482661,-0.994623,0.485119,0.384334,-0.835923,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
645256,0.191253,-1.340601,0.485119,-1.717718,-0.835923,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
308367,0.191253,1.115843,0.485119,0.384334,-0.835923,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [74]:
# Ordena las columnas alfabéticamente por nombre.
df1 = df1[sorted(df1.columns)]

# Asigna las características independientes a X
X = X_features_final

# Extrae la columna objetivo y la asigna a la variable y
y = df1[y_target]

#Verificar Forma de X y y
X.shape, y.shape

((692500, 74), (692500,))

# <font color=4CBB17>**Modelo Ramdom Forest**</font>


## <font color=FF7F50>**Selección de variables con método Integrado** </font>

>Se utiliza el método Lasso porque combina selección de características y regularización mediante penalización L1, que permite eliminar variables irrelevantes al forzar coeficientes a cero y evita el sobreajuste reduciendo la magnitud de los coeficientes

In [75]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

# Selector de variables con Lasso.....se recompueden probar varios alpha....https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
sel_ = SelectFromModel(Lasso(alpha=0.1, max_iter=10000), max_features=10)  # Se ajusta alpha según sea necesario
sel_.fit(X, y)
print(sel_.estimator_.coef_)

#Obtener variables seleccionadas
X_new = sel_.get_support()#descarta los coeficientes mas cercanos a 0

df_newlasso = X.iloc[:,X_new]
df_newlasso.head()

[ 0.1297692  -0.          0.10890142  0.         -0.07897463  0.
  0.          0.         -0.          0.         -0.         -0.
  0.         -0.          0.         -0.          0.          0.
  0.         -0.          0.         -0.         -0.          0.
 -0.          0.         -0.          0.         -0.          0.
 -0.         -0.         -0.          0.         -0.          0.
 -0.          0.          0.          0.          0.          0.
 -0.          0.          0.         -0.         -0.          0.
 -0.          0.         -0.          0.         -0.          0.
  0.         -0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.         -0.         -0.         -0.
 -0.          0.          0.          0.         -0.         -0.
  0.         -0.        ]


,ESTU_VALORMATRICULAUNIVERSIDAD,FAMI_ESTRATOVIVIENDA,ESTU_PAGOMATRICULAPROPIO
ID,,,
904256,1.482661,0.485119,-0.835923
645256,0.191253,0.485119,-0.835923
308367,0.191253,0.485119,-0.835923
470353,0.836957,1.240860,-0.835923
989032,0.191253,0.485119,-0.835923


> El método integrado dió como resultado las variables númericas, por lo tanto se entrana el modelo con estas

In [53]:
# Libreria
from sklearn.linear_model import LogisticRegressionCV

In [56]:
# Entrenamiento del modelo de Random Forest sin hiperparametros
rf_model = RandomForestClassifier()

## <font color=FF7F50>**Cross Validation** </font>


In [59]:
from sklearn.model_selection import cross_validate

# Deficición de la función para CV
def cross_validation_accuracy(model, X, y, cv=5):
    """Función para realizar validación cruzada y devolver únicamente métricas de accuracy."""
    results = cross_validate(estimator=model,
                             X=X,
                             y=y,
                             cv=cv,
                             scoring='accuracy',
                             return_train_score=True)

# En este caso solo nos interesa accuracy, pero se pueden agregar mas metricas como recall o F1-score
    return {
        "Training Accuracy scores": results['train_score'],
        "Mean Training Accuracy": results['train_score'].mean() * 100,
        "Validation Accuracy scores": results['test_score'],
        "Mean Validation Accuracy": results['test_score'].mean() * 100
    }

In [60]:
# Realizar validación cruzada con el conjunto de características seleccionadas
cv_results = cross_validation_accuracy(rf_model, df_newlasso, y)

In [61]:
# Imprimir los resultados de la validación cruzada
print("Cross Validation Results:")
for metric, score in cv_results.items():
    print(f"{metric}: {score}")

Cross Validation Results:
Training Accuracy scores: [0.37570939 0.37634838 0.37559386 0.37603791 0.37570578]
Mean Training Accuracy: 37.58790613718411
Validation Accuracy scores: [0.37584116 0.37345126 0.37639711 0.37454152 0.37606498]
Mean Validation Accuracy: 37.525920577617335


## <font color=FF7F50> **Selección Hiperparámetros : Random Search**</font>

La información para los hiper se tomó de : https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74. En base a esto se realizó la grilla

In [62]:
# Definicion de hiperparametros para bosques aleatorios
params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True]
}

In [63]:
# Crear el modelo de Random Forest
rf_model = RandomForestClassifier()

In [66]:
# La busqueda se realiza de manera sencilla, no muy exhaustiva
from sklearn.model_selection import RandomizedSearchCV

# Definir el RandomizedSearchCV con validación cruzada
random_search = RandomizedSearchCV(estimator = rf_model,
                           param_distributions = params,
                           n_iter=10, # Numero de iteraciones
                           cv = 3,  # Número de folds para validación cruzada
                           scoring = 'accuracy',  # Se puede cambiar por otra métrica si se desea
                           n_jobs = -1,  # Usar todos los núcleos disponibles para acelerar la búsqueda
                           verbose = 2)

In [67]:
# Ajustar el modelo al conjunto de entrenamiento
random_search.fit(df_newlasso, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [10, 20, None],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [100, 200]},
                   scoring='accuracy', verbose=2)

### <font color=1E90FF> **Mejores Parametros encontrados**

In [ ]:
# Imprimir los mejores parámetros encontrados
print("Best Parameters: ", random_search.best_params_)

Best Parameters:  {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


**Modelo Optimizado**

In [68]:
# Obtener el mejor modelo
best_rf_model = random_search.best_estimator_
best_rf_model


RandomForestClassifier(min_samples_split=5)

**Validación cruzada de  modelo optimizado**

In [69]:
# Realizar validación cruzada con el modelo optimizado
cv_results_optimized = cross_validation_accuracy(best_rf_model, df_newlasso, y)


In [70]:
# Imprimir los resultados de la validación cruzada del modelo optimizado
print("Cross Validation Results (Optimized Random Forest):")
for metric, score in cv_results_optimized.items():
    print(f"{metric}: {score}")

Cross Validation Results (Optimized Random Forest):
Training Accuracy scores: [0.37570939 0.37634657 0.37559567 0.37603791 0.37570578]
Mean Training Accuracy: 37.58790613718411
Validation Accuracy scores: [0.37613718 0.37347292 0.37657762 0.37454152 0.37607942]
Mean Validation Accuracy: 37.53617328519855
